In [1]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
from keras.optimizers import SGD, Adadelta, Adagrad
from keras.regularizers import l1, l2, l1_l2
from keras.callbacks import EarlyStopping

Using Theano backend.


In [3]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 1000)
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

In [4]:
train = pd.read_pickle('papa_train.pkl')
test = pd.read_pickle('papa_test.pkl')

In [8]:
target = train['Responders']

In [9]:
feature_names = [f for f in train.columns if f not in ['UCIC_ID','Responders']]

In [10]:
# model architechture
def keras_model(train):
    
    input_dim = train.shape[1]
    classes = 2
    
    model = Sequential()
    model.add(Dense(100, activation = 'relu', input_shape = (input_dim,))) #layer 1
    model.add(Dense(30, activation = 'relu')) #layer 2
    model.add(Dense(classes, activation = 'sigmoid')) #output
    model.compile(optimizer = 'adam', loss='binary_crossentropy')
    return model

callback = EarlyStopping(monitor='val_acc',patience=3)

In [11]:
X_train, X_valid, y_train, y_valid = train_test_split(train, target, test_size = 0.3, stratify = target)

In [12]:
ky_train = np_utils.to_categorical(np.array(y_train))
ky_valid = np_utils.to_categorical(np.array(y_valid))

In [13]:
nb_epoch = 120
nb_batch_size = 50

In [14]:
X_train = np.matrix(X_train[feature_names])
X_valid = np.matrix(X_valid[feature_names])

In [15]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

In [16]:
model = keras_model(X_train)

In [17]:
model.fit(X_train, ky_train, batch_size=32, epochs=2, callbacks=[early_stopping], validation_data=(X_valid, ky_valid))

Train on 210000 samples, validate on 90000 samples
Epoch 1/2
210000/210000 [==============================] - 14s - loss: 0.3851 - val_loss: 0.3663

In [26]:
preds = model.predict_proba(np.matrix(test[feature_names]))[:,1]

194688/200000 [============================>.] - ETA: 0s 

In [28]:
sub = pd.read_csv('sample_submission_fBo3EW5.csv')
sub['UCIC_ID'] = test['UCIC_ID']
sub['Responders'] = preds

In [29]:
sub.to_csv('submissions/keras1.csv',index=False) ## 0.4567

In [30]:
def build_model(tr):
    
    input_dim = tr.shape[0]
    classes = 2
    
    model = Sequential()
    model.add(Dense(64, input_shape = (input_dim, )))
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    model.add(Dense(30, init = 'glorot_uniform', W_regularizer = l1(1e-6)))
    model.add(Activation('relu'))
    model.add(Dropout(0.4))

    model.add(Dense(classes))
    model.add(Activation('sigmoid'))
    model.compile(loss = 'binary_crossentropy', optimizer='adagrad')
    return model

callback = EarlyStopping(monitor='val_acc',patience=3)

In [47]:
model = keras_model(X_train)

In [ ]:
model.fit(X_train, ky_train, batch_size=32, epochs=nb_epoch, callbacks=[early_stopping], validation_data=(X_valid, ky_valid))

Train on 210000 samples, validate on 90000 samples
Epoch 1/120
210000/210000 [==============================] - 16s - loss: 0.3851 - val_loss: 0.3644

In [26]:
preds = model.predict_proba(np.matrix(test[feature_names]))[:,1]

194688/200000 [============================>.] - ETA: 0s 